In [1]:
from QAOAUtils import *
from PSC import *
import pickle as pkl
from qiskit_algorithms.optimizers import COBYLA

(CVXPY) Oct 29 07:40:04 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


In [2]:
DATA = pkl.load( open ("PortfolioOptDepth0Test.p", "rb") )
stock_data = pkl.load( open ("stock_data_depth0.p", "rb") )

In [3]:
idx_list = np.linspace(0,999,1000,dtype='int')

PO_A_list = []
stock_data_list = []
PO_init_list = []
results = {
    "VDISTANCE": [],
    "DISTANCE": [],
}

for i in idx_list:
    PO_A_list.append(DATA[3][i])
    PO_init_list.append(DATA[2][i])
    stock_data_list.append(stock_data[i])

In [4]:
A = PO_A_list[0]
PO_data = stock_data_list[0]
v = PO_solve(PO_data["mu"], PO_data["sigma"],B=10)
round_v = np.round(v)
dist = np.linalg.norm(v-round_v)

In [5]:
for _,A in tqdm(enumerate(PO_A_list)):
    PO_data = stock_data_list[_]
    v = PO_solve(stock_data_list[_]["mu"],stock_data_list[_]["sigma"], B=10)
    v_round = round_v
    results["VDISTANCE"].append(np.linalg.norm(v-v_round))

0it [00:00, ?it/s]

In [6]:
for A in tqdm(PO_A_list):
    t=[map_reduce(v) for v in brute_force_maxcut(A)[0]]
    results["DISTANCE"].append(np.min([np.linalg.norm(v-i) for i in t]))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
with open("DistancesPO.p", "wb") as file:
    pkl.dump(results, file)